In [1]:
import gyoto
import gyoto.std
import gyoto.core
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
kpc_meter=3.08568025e19
c_SI=299792458.
me_SI=9.10938188e-28
kb_SI=1.3806504e-23

In [3]:
metric = gyoto.std.KerrBL()
metric.spin(0.)
metric.mass(4.3e6,"sunmass")

In [4]:
fov = 200 #muas
npix = 128
incli = 20 #deg

screen = gyoto.Screen()
screen.metric(metric)
screen.distance(8.3,"kpc")
screen.time(8.3,"kpc")
screen.fieldOfView(fov,"µas")
screen.inclination(180.-incli,"°")
screen.PALN(180,"°")
screen.resolution(npix)
spectro=gyoto.core.Spectrometer('freqlog')
spectro.set("NSamples", 1)
spectro.set("Band", (math.log10(230e9), math.log10(230e9)), "Hz")
screen.spectrometer(spectro)

In [5]:
disk = gyoto.std.ThickDisk()
disk.thickDiskInnerRadius(6.)
disk.thickDiskZGaussianSigma(0.3)
disk.numberDensityAtInnerRadius(6e6,"cm-3")
disk.temperatureAtInnerRadius(200.*me_SI*c_SI*c_SI/kb_SI)
disk.temperatureSlope(-0.84)
disk.densitySlope(-1.5)
disk.metric(metric)
disk.opticallyThin(True)
disk.magneticConfiguration('Vertical')
disk.rMax(100)
disk.deltaInObj(0.05)

In [6]:
scry = gyoto.Scenery()
scry.metric(metric)
scry.screen(screen)
scry.astrobj(disk)
scry.tMin(-1e10)
scry.nThreads(4)
scry.parallelTransport(True)
scry.relTol(1.e-10)
scry.absTol(1.e-10)

scry.requestedQuantitiesString("Spectrum SpectrumStokesQ SpectrumStokesU SpectrumStokesV")

In [ ]:
image_cube=scry.rayTrace()

SI2Jy = (fov*1e-6/3600*np.pi/180./npix)**2*1e26

imgI=image_cube["Spectrum"][0,:,:]*SI2Jy

imgQ=image_cube["SpectrumStokesQ"][0,:,:]*SI2Jy

imgU=image_cube["SpectrumStokesU"][0,:,:]*SI2Jy

imgV=image_cube["SpectrumStokesV"][0,:,:]*SI2Jy

In [ ]:
vmin = min(imgQ.min(), imgU.min())
vmax = max(imgQ.max(), imgU.max())
scale_LP = max(abs(vmin), abs(vmax))
normLP = colors.Normalize(vmin=-scale_LP, vmax=scale_LP)

scale_CP = max(abs(imgV.min()), abs(imgV.max()))
normCP = colors.Normalize(vmin=-scale_CP, vmax=scale_CP)

In [ ]:
fig, axs = plt.subplots(1, 4)
fig.set_size_inches(16, 9)
fig.tight_layout()


axs[0].tick_params(axis='both', which='major', labelsize=16)
for ii in range(1,4):
    axs[ii].set_xticks([])
    axs[ii].set_yticks([])

cmap1 = 'afmhot'
cmap2 = "seismic"

img_Gyoto_I = axs[0].imshow(imgI, interpolation='nearest', cmap=cmap1, extent=(-fov/2.,fov/2.,-fov/2.,fov/2.))
axs[0].set_title("Stokes I", fontsize=22)
img_Gyoto_Q = axs[1].imshow(imgQ, interpolation='nearest', cmap=cmap2, norm=normLP, extent=(-fov/2.,fov/2.,-fov/2.,fov/2.))
axs[1].set_title("Stokes Q", fontsize=22)
img_Gyoto_U = axs[2].imshow(imgU, interpolation='nearest', cmap=cmap2, norm=normLP, extent=(-fov/2.,fov/2.,-fov/2.,fov/2.))
axs[2].set_title("Stokes U", fontsize=22)
img_Gyoto_V = axs[3].imshow(imgV, interpolation='nearest', cmap=cmap2, norm=normCP, extent=(-fov/2.,fov/2.,-fov/2.,fov/2.))
axs[3].set_title("Stokes U", fontsize=22)

cb_imgI = fig.colorbar(img_Gyoto_I, ax=axs[0], location='bottom', pad=0.05)
cb_imgI.set_label('Jy', fontsize=18)
cb_imgI.ax.tick_params(axis='both', which='major', labelsize=16)
cb_imgLP = fig.colorbar(img_Gyoto_Q, ax=axs[1:3], location='bottom', pad=0.05)
cb_imgLP.set_label('Jy', fontsize=18)
cb_imgLP.ax.tick_params(axis='both', which='major', labelsize=16)
cb_imgCP = fig.colorbar(img_Gyoto_V, ax=axs[3], location='bottom', pad=0.05)
cb_imgCP.set_label('Jy', fontsize=18)
cb_imgCP.ax.tick_params(axis='both', which='major', labelsize=16)

In [ ]:
plt.figure(2, figsize=(16.,16.))
plt.clf()
plt.ion()
plt.gca().invert_xaxis()
plt.xlabel("X [$\mu$as]", fontsize=20.)
plt.ylabel("Y [$\mu$as]", fontsize=20.)
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)

imgEVPA=np.arctan2(imgU, imgQ)/2.
imgLP=np.sqrt(imgQ**2 +imgU**2)/imgI
imgCP = imgV/imgI

muasperpxl=fov/npix
ntick=31 # nb of ticks to plot along horiz dir
steps=int(npix/ntick)
multmarker=2. # size of polar arrow in plot
for itick in range(ntick):
    for jtick in range(ntick):
        imid=int(steps*itick+steps/2.)
        jmid=int(steps*jtick+steps/2.)
        Iloc=imgI[jmid,imid]
        EVPAloc=imgEVPA[jmid,imid]
        LPloc=imgLP[jmid,imid]
        xx=(imid-int(npix/2.))*muasperpxl
        yy=(jmid-int(npix/2.))*muasperpxl
        if imgI[jmid,imid]>0.08*imgI.max():
            plt.plot([xx+multmarker*LPloc*math.sin(EVPAloc),xx-multmarker*LPloc*math.sin(EVPAloc)],
                     [yy-multmarker*LPloc*math.cos(EVPAloc),yy+multmarker*LPloc*math.cos(EVPAloc)],
                     color='white', linewidth=1.5)

plt.imshow(imgI, interpolation='nearest', origin='lower', cmap=cmap1, extent=(-fov/2.,fov/2.,-fov/2.,fov/2.))